In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from compmec import nurbs
from helper import getH, getD, solve_system

# Introdução

No notebook anterior tratamos EDPs de diferentes tipos, todos lineares. Mas a mema abordagem utilizada não funciona para o problema que trataremos aqui: resolver a Equação de Navier Stokes.

Antes disso, introduzimos ao problema. Teremos as quantidades:

* Velocidade do fluido: $\mathbf{u} = \left(u_1, \ u_2, \ u_3\right) = \left(u, \ v, \ w\right)$
* Pressão do fluido: $p$
* Fator de campo: $\mathbf{g}$
* Densidade: $\rho$
* Fluido newtoniano de viscosidade: $\mu$
* Tempo: $t$
* Espaço: $\left(x, \ y, \ z\right) = \left(x_1, \ x_2, \ x_3\right)$

Resolver a equação de Navier Stokes se resume a encontrar os valores da velocidade $\mathbf{u}$ e pressão $p$ ao longo do domínio $\Omega_{xyzt}$.

Na notação vetorial temos a equação de balanço de massa dado por $(1.1)$ e o balanço de momento dado por $(1.2)$

$$\dfrac{\partial \rho}{\partial t} + \nabla \cdot \left(\rho \mathbf{u}\right) = 0 \tag{1.1}$$

$$\dfrac{\partial \left(\rho \mathbf{u}\right)}{\partial t} + \nabla \cdot \left( \rho \mathbf{u} \otimes \mathbf{u} \right) = -\nabla p + \rho \mathbf{g} + \mu \nabla^2 \mathbf{u}\tag{1.2}$$

Na notação indicial, $(1.1)$ e $(1.2)$ se tornam $(1.3)$ e $(1.4)$

$$\dfrac{\partial \rho}{\partial t} + \sum_{i = 1}^{3} \dfrac{\partial \left(\rho u_i\right)}{\partial x_i} = 0
\tag{1.3}$$
$$\dfrac{\partial \rho u_i}{\partial t} + \sum_{j=1}^{3} \dfrac{\partial \left(\rho u_i u_j\right)}{\partial x_j} = -\dfrac{\partial p}{\partial x_i} + \rho g_i + \mu \sum_{j=1}^{3} \dfrac{\partial^2 u_i}{\partial x_j^2} \ \ \ \ \ \forall i \tag{1.4}$$

Para o primeiro exemplo, simplificaremos e utilizaremos:

* Malha bidimensional: $\vec{u} = (u, \ v, \ 0)$
* Fluido incompressível: $\rho = \text{const}$
* Força de campo nulas: $\mathbf{g} = \mathbf{0}$.

Também fazemos a adimensionalização do problema, resultando em $(1.5)$ e $(1.6)$:

$$\begin{align*}\nabla \cdot \vec{u} & = 0 \tag{1.5} \\ \dfrac{\partial \vec{u}}{\partial t} + \left(\vec{u} \cdot \nabla\right) \vec{u} & = -\nabla p + \mu \nabla^2 \vec{u} \tag{1.6} 
\end{align*}$$

Aqui chamaremos de $\mu$ como o inverso do número de Reynolds, apenas para facilitar a notação. Não confundir com o $\mu$ da viscosidade da Equação $(1.2)$.




Na malha $\Omega_{xyt} = \left[0, \ 1\right] \times \left[0, \ 1\right] \times \left[0, \ \infty\right)$.

# Diferentes formulações

Embora tenhamos já apresentado as equações, há diferentes de tipos de formulação com diferentes variáveis. Antes de mostra-las, vamos enunciar funções auxiliares:

* Linhas de corrente $\psi$ 

   Assumindo que as velocidades $u$ e $v$ satisfazem

   $$u = \dfrac{\partial \psi}{\partial y} \ \ \ \ \ \ \ \ \ \ \ \ \ v =  -\dfrac{ \partial  \psi}{ \partial x} \tag{2.1}$$
   $$\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} = \dfrac{\partial}{\partial x}\left(\dfrac{\partial \psi}{\partial y}\right) + \dfrac{\partial}{\partial y}\left(-\dfrac{\partial \psi}{\partial x}\right) = 0$$

   Então adotar $\psi$ já satifaz a Equação $(1.5)$

* Vorticidade $w$:

   Como a Equação $(1.6)$ envolve o gradiente da pressão, e que é um dos grandes problemas em fluidos, uma solução é elimina-lo tomando o rotacional da Equação $(1.6)$.

   $$w = \nabla \times \vec{u} = \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y} \tag{2.2}$$
   $$\dfrac{\partial w}{\partial t} + \left(u \cdot \nabla \right) w = \mu \nabla^2 w \tag{2.3}$$

1. Formulação de variáveis $(u, \ v, \ p)$

   A primeira delas é utilizar as variáveis padrões, que expandindo as Equações $(1.5)$ e $(1.6)$ obteremos: 

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\ \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right)\\ \dfrac{\partial v}{\partial t} + u\dfrac{\partial v}{\partial x} + v\dfrac{\partial v}{\partial y} & = -\dfrac{\partial p}{\partial y} + \mu \left(\dfrac{\partial^2 v}{\partial x^2}+\dfrac{\partial^2 v}{\partial y^2}\right)  \end{align*}$$

   Nessa formulação temos $3$ equações e $3$ variáveis a serem descobertas.

2. Formulação de variáveis $(\psi, \ p)$

   Satisfaz-se automaticamente a Equação $(1.5)$ de forma que reformulamos o sistema para:

   $$\begin{align*}\dfrac{\partial^2 \psi }{\partial t \partial y} + \dfrac{\partial \psi}{\partial y} \dfrac{\partial^2 \psi}{\partial x \partial y} -\dfrac{\partial \psi}{\partial x} \dfrac{\partial^2 \psi}{\partial y^2} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^3 \psi}{\partial x^2 \partial y}+\dfrac{\partial^3 \psi}{\partial y^3}\right)  \\ -\dfrac{\partial^2 \psi}{\partial t \partial x} - \dfrac{\partial \psi}{\partial y}\dfrac{\partial^2 \psi}{\partial x^2} + \dfrac{\partial \psi}{\partial x}\dfrac{\partial^2 \psi }{\partial x \partial y} & = -\dfrac{\partial p}{\partial y} - \mu \left(\dfrac{\partial^3 \psi}{\partial x^3}+\dfrac{\partial^3 \psi}{\partial x\partial y^2}\right)\end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

3. Formulação de variáveis $(u, \ v, \ w)$

   Fazendo o rotacional ($\nabla \times$) da Equação $(1.6)$, juntando com a Equação $(1.5)$ e $(2.2)$ teremos o sistema

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\ w & = \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}\\ \dfrac{\partial w}{\partial t} + u \dfrac{\partial w}{\partial x} + v\dfrac{\partial w}{\partial y} & = \mu \left(\dfrac{\partial^2 w}{\partial x^2}+\dfrac{\partial^2 w}{\partial y^2}\right)\\  \end{align*}$$

   Nessa formulação temos $3$ equações e $3$ variáveis a serem descobertas.

3. Formulação de variáveis $(u, \ v)$

   Fazendo o rotacional ($\nabla \times$) da Equação $(1.6)$, juntando com a Equação $(1.5)$ e $(2.2)$ teremos o sistema

   $$\begin{align*}\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \\  \dfrac{\partial }{\partial t}\left(\dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y} \right) + \text{non-linear} & = \mu \cdot \text{diffusion}  \end{align*}$$

   Com

   $$\begin{align*}\text{diffusion} & = \dfrac{\partial^3 v}{\partial x^3}-\dfrac{\partial^3 u}{\partial x^2 \partial y}+\dfrac{\partial^3 v}{\partial x\partial y^2}-\dfrac{\partial^3 u}{\partial y^3} \\ \text{non-linear} & = u \dfrac{\partial^2 v}{\partial x^2} - u \dfrac{\partial^2 u}{\partial x \partial y} + v\dfrac{\partial^2 v}{\partial x\partial y} - v\dfrac{\partial^2 u}{\partial y^2}  \\ \end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

5. Formulação de variáveis $(\psi, \ w)$

   Do mesmo modo que a formulação 3, mas substituimos $u$ e $v$ pelos da Equação $(2.1)$, resultando em

   $$\begin{align*}w + \dfrac{\partial^2 \psi}{\partial x^2} + \dfrac{\partial^2 \psi}{\partial y^2} & = 0 \\ \dfrac{\partial w}{\partial t} + \dfrac{\partial \psi}{\partial y} \dfrac{\partial w}{\partial x} - \dfrac{\partial \psi}{\partial x} \dfrac{\partial w}{\partial y} & = \mu \left(\dfrac{\partial^2 w}{\partial x^2}+\dfrac{\partial^2 w}{\partial y^2}\right)\\  \end{align*}$$

   Nessa formulação temos $2$ equações e $2$ variáveis a serem descobertas.

6. Formulação na variável $(\psi)$

   Pegando a formulação 5, basta substituir a primeira equação na segunda para obter

   $$\dfrac{\partial}{\partial t}\left(\dfrac{\partial^2 \psi}{\partial x^2}+ \dfrac{\partial^2 \psi}{\partial y^2}\right) = \text{non-linear} + \mu \cdot \text{diffusion}$$

   Com 

   $$\begin{align*}\text{diffusion} & = \dfrac{\partial^4 \psi}{\partial x^4} + 2\dfrac{\partial^4 \psi}{\partial x^2 \partial y^2} + \dfrac{\partial^4 \psi}{\partial y^4} \\ \text{non-linear} & = \dfrac{\partial \psi}{\partial x} \dfrac{\partial}{\partial y}\left(\dfrac{\partial^2 \psi}{\partial x^2}+\dfrac{\partial^2 \psi}{\partial y^2}\right) - \dfrac{\partial \psi}{\partial y}\dfrac{\partial}{\partial x}\left(\dfrac{\partial^2 \psi}{\partial x^2}+\dfrac{\partial^2 \psi}{\partial y^2}\right) \\ & =  \dfrac{\partial \psi}{\partial x} \dfrac{\partial^3 \psi}{\partial y^3} - \dfrac{\partial \psi}{\partial y}\dfrac{\partial^3 \psi}{\partial x \partial y^2} + \dfrac{\partial \psi}{\partial x} \dfrac{\partial^3 \psi}{\partial x^2\partial y} -\dfrac{\partial \psi}{\partial y}\dfrac{\partial^3 \psi}{\partial x^3}\end{align*}$$

   
   Nessa formulação temos apenas $1$ equação e $1$ variáveis a ser descoberta.